In [ ]:
import os
from typing import Any, Callable, Dict, List, Sequence, Tuple

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

from flavor.serve.apps import InferAPP
from flavor.serve.inference.data_models.api import (
    BaseAiCOCOImageInputDataModel,
    BaseAiCOCOImageOutputDataModel,
)
from flavor.serve.inference.data_models.functional import AiImage
from flavor.serve.inference.inference_models import BaseAiCOCOHybridInferenceModel
from flavor.serve.inference.strategies import AiCOCOClassificationOutputStrategy

In [ ]:
class HybridNet(nn.Module):
    def __init__(self, csv_input_dim, image_feature_dim, fusion_hidden_dim=128):
        """
        Args:
            csv_input_dim (int): csv dimension
            image_feature_dim (int): image feature dimension
            fusion_hidden_dim (int): fusion hidden dimension
        """
        super(HybridNet, self).__init__()
        
        # image branch
        self.image_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.image_fc = nn.Linear(64, image_feature_dim)
        
        # tabular branch
        self.csv_fc = nn.Linear(csv_input_dim, image_feature_dim)
        
        # fusion branch
        self.fusion_fc = nn.Linear(2 * image_feature_dim, fusion_hidden_dim)
        
        # classifier
        self.out = nn.Linear(fusion_hidden_dim, 2)
        
    def forward(self, tensor_image, tensor_tabular):
        """
        Args:
            tensor_image (tensor)
            tensor_tabular (tensor)
        Returns:
            logits (tensor)
        """
        # image forward
        x_img = self.image_conv(tensor_image)         # shape: (batch_size, 64, 1, 1)
        x_img = x_img.view(x_img.size(0), -1)    # shape: (batch_size, 64)
        x_img = self.image_fc(x_img)             # shape: (batch_size, image_feature_dim)
        
        # tabular forward
        x_csv = self.csv_fc(tensor_tabular)          # shape: (batch_size, image_feature_dim)
        
        # fusion
        x = torch.cat((x_img, x_csv), dim=1)     # shape: (batch_size, 2 * image_feature_dim)
        x = F.relu(self.fusion_fc(x))
        logits = self.out(x)
        return logits


In [ ]:
class ClassificationInferenceModel(BaseAiCOCOHybridInferenceModel):
    def __init__(self):
        self.formatter = AiCOCOClassificationOutputStrategy()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        super().__init__()

    def define_inference_network(self) -> Callable:
        network = HybridNet(csv_input_dim=22, image_feature_dim=32)
        network.eval()
        network.to(self.device)
        return network

    def set_categories(self) -> List[Dict[str, Any]]:
        categories = [{"name": str(i)} for i in range(2)]
        return categories

    def set_regressions(self) -> None:
        return None

    def data_reader(self, image_files: Sequence[str], table_files: Sequence[str]):
        image = Image.open(image_files[0])
        tabular = pd.read_csv(table_files[0])
        return image, tabular

    def preprocess(self, x):
        image_data, table_data = x
        # image data
        img = np.array(image_data).transpose(2, 0, 1).astype(np.float32)
        tensor_image = torch.tensor(img).unsqueeze(0).to(self.device)
        
        # csv data
        tabular = table_data
        tensor_tabular = torch.tensor(tabular.values.astype(np.float32)).to(self.device)
        
        return tensor_image, tensor_tabular

    def inference(self, x) -> torch.Tensor:
        tensor_image, tensor_tabular = x
        with torch.no_grad():
            out = self.network(tensor_image, tensor_tabular)
        return out

    def postprocess(self, model_out: torch.Tensor, **kwargs) -> np.ndarray:
        model_out = model_out.squeeze(0).cpu().detach()
        model_out = (nn.functional.softmax(model_out, dim=0) > 0.4).long()
        return model_out.numpy()

    def output_formatter(
        self,
        model_out: np.ndarray,
        images: Sequence[AiImage],
        categories: Sequence[Dict[str, Any]],
        **kwargs
    ) -> BaseAiCOCOImageOutputDataModel:

        output = self.formatter(model_out=model_out, images=images, categories=categories)
        return output

In [ ]:
# This block is only for jupyter notebook. You don't need this in stand-alone script.
import nest_asyncio
nest_asyncio.apply()

In [ ]:
app = InferAPP(
    infer_function=ClassificationInferenceModel(),
    input_data_model=BaseAiCOCOImageInputDataModel,
    output_data_model=BaseAiCOCOImageOutputDataModel,
)

In [ ]:
app.run(port=int(os.getenv("PORT", 9111)))

### Send request
We can send request to the running server by `send_request.py` which opens the input files and the corresponding JSON file and would be sent via formdata. We expect to have response in AiCOCO format.

```bash
# pwd: examples/inference
python send_request.py -f test_data/hybrid/451c164d-7684-44b1-81b2-956247db765b_20160112_102927.jpg -f test_data/hybrid/test_cls.csv -d test_data/hybrid/input.json
```

## Setup Dockerfile
In order to interact with other services, we have to wrap the inference model into a docker container. Here's an example of the dockerfile.

```dockerfile
FROM nvidia/cuda:12.2.2-runtime-ubuntu20.04

RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        python3\
        python3-pip \
    && ln -sf /usr/bin/python3 /usr/bin/python

RUN pip install torch==2.1.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121 --default-timeout=1000
RUN pip install https://github.com/ailabstw/FLaVor/archive/refs/heads/release/stable.zip

WORKDIR /app

COPY your_script.py  /app/

CMD ["python", "your_script.py"]

```